In [ ]:
def createPeriodicSplines( X, Y ):
    tck, u = interpolate.splprep([X, Y], s=0, per=1)
    return tck

def createSplines( X, Y ):
    tck, u = interpolate.splprep([X, Y], s=0 )
    return tck

def computeNodeLocations( N, tck ):
    t = np.linspace( 0, 1, N )
    t = np.split( t, 2 )
    t[0] = -np.cos( t[0] * pi*2 )/4 + 1/4
    t[1] = np.cos( t[1] * pi*2 )/4 + 3/4
    t = np.concatenate( ( t[0], t[1] ) )
    x, y = interpolate.splev(t, tck)
    return x, y

def computeNormParameterizationDerivative( N, tck ):
    t = np.linspace( 0, 1, N )
    t = np.split( t, 2 )
    t[0] = -np.cos( t[0] * pi*2 )/4 + 1/4
    t[1] = np.cos( t[1] * pi*2 )/4 + 3/4
    t = np.concatenate( ( t[0], t[1] ) )
    dx_dt, dy_dt = interpolate.splev(t, tck, der=1)
    ds_dt = np.sqrt( dx_dt**2 + dy_dt**2 )
    return ds_dt

## this function I think is unnecessary but not sure
def computeRadial( N, tck ):
    t = np.linspace( 0, 1, N )
    t = np.split( t, 2 )
    t[0] = -np.cos( t[0] * pi*2 )/4 + 1/4
    t[1] = np.cos( t[1] * pi*2 )/4 + 3/4
    t = np.concatenate( ( t[0], t[1] ) )
    rx, ry = interpolate.splev(t, tck)
    norm = np.sqrt( rx**2 + ry**2 )
    return rx/norm, ry/norm

def computeTangent( N, tck ):
    t = np.linspace( 0, 1, N )
    t = np.split( t, 2 )
    t[0] = -np.cos( t[0] * pi*2 )/4 + 1/4
    t[1] = np.cos( t[1] * pi*2 )/4 + 3/4
    t = np.concatenate( ( t[0], t[1] ) )
    tx, ty = interpolate.splev(t, tck, der=1)
    norm = np.sqrt( tx**2 + ty**2 )
    return tx/norm, ty/norm

def computeNormal( N, tck ):
    t = np.linspace( 0, 1, N )
    t = np.split( t, 2 )
    t[0] = -np.cos( t[0] * pi*2 )/4 + 1/4
    t[1] = np.cos( t[1] * pi*2 )/4 + 3/4
    t = np.concatenate( ( t[0], t[1] ) )
    nx, ny = interpolate.splev(t, tck, der=2)
    norm = -np.sqrt( nx**2 + ny**2 )
    return nx/norm, ny/norm

def computeBinormal( N, tck ):
    tx, ty = computeTangent( N, tck )
    nx, ny = computeNormal( N, tck )
    tangentVector = np.vstack( (tx, ty) )
    normalVector = np.vstack( (nx, ny) )
    binormal = np.cross( np.transpose(tangentVector), np.transpose(normalVector) )
    return -binormal

def computeCurvature( N, tck ):
    t = np.linspace( 0, 1, N )
    t = np.split( t, 2 )
    t[0] = -np.cos( t[0] * pi*2 )/4 + 1/4
    t[1] = np.cos( t[1] * pi*2 )/4 + 3/4
    t = np.concatenate( ( t[0], t[1] ) )
    d1x, d1y = interpolate.splev(t, tck, der=1)
    d2x, d2y = interpolate.splev(t, tck, der=2)
    curvature = (d2x*d1y - d2y*d1x) / (d1x**2+d1y**2)**(3/2)
    return curvature

def computeChordLength( X ):
    return np.max(X) - np.min(X)

def computeAirfoilPseudoThickness( Y ):
    return np.max(Y) - np.min(Y)

def createWakeGeometry( X, Y, AOA ):
    chordLength = computeChordLength( X )
    wakeLength = 1000 * chordLength
    x0 = .5 * ( X[0] + X[-1] )
    y0 = .5 * ( Y[0] + Y[-1] )
    xf = x0 + wakeLength * np.cos( AOA * pi/180 )
    yf = y0 + wakeLength * np.sin( AOA * pi/180 )
    Xw = np.asarray( [x0, xf] )
    Yw = np.asarray( [y0, yf] )
    tck, u = interpolate.splprep([Xw, Yw], k=1, s=0, quiet=True )
    return tck

def computeWakeNormal( Nw, AOA ):
    nxw = -np.sin( AOA * pi/180 ) * np.ones( Nw )
    nyw = np.cos( AOA * pi/180 ) * np.ones( Nw )
    return nxw, nyw

def ComputeAlpha( x1, y1, x2, y2 ):
    return atan2( -(y1-y2), -(x1-x2) )

def GlobalToPanelCoords( x, y, x1, y1, x2, y2 ):
    alpha = ComputeAlpha( x1, y1, x2, y2 )
    
    ## first displace all points such that the coordinate system origin is on (x1, y1)
    xp = cos(alpha) * ( x - x1 ) + sin(alpha) * ( y - y1 )
    yp = -sin(alpha) * ( x - x1 ) + cos(alpha) * ( y - y1 )
    xp1 = 0 
    yp1 = 0 
    xp2 = cos(alpha) * ( x2 - x1 ) + sin(alpha) * ( y2 - y1 )
    yp2 = -sin(alpha) * ( x2 - x1 ) + cos(alpha) * ( y2 - y1 )
    
    return xp, yp, xp1, yp1, xp2, yp2, alpha
    